# CV-2
# ISHANT KUNDRA

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

## Q1. Import and Understand the data [7 Marks]


### A. Import and read ‘images.npy’. [1 Marks]

In [ ]:
images_npy=np.load("Desktop/data/images.npy", allow_pickle=True)

In [ ]:
images = images_npy[:,0]
mask = images_npy[:,1]


In [ ]:
plt.imshow(images[200])


In [ ]:
plt.imshow(images[302])


In [ ]:
print(images[302].shape)


In [ ]:
images_npy[0][1]

In [ ]:
mask[302] 


In [ ]:
img_w= 224
img_h = 224
alpha= 1

### B. Split the data into Features(X) & labels(Y). Unify shape of all the images. [3 Marks] 

In [ ]:
import cv2
from tensorflow.keras.applications.mobilenet import preprocess_input

masks = np.zeros((int(images_npy.shape[0]),img_h,img_w))
xtrain = np.zeros((int(images_npy.shape[0]),img_h,img_w,3))
for index in range(images_npy.shape[0]):
    image_n = images_npy[index][0]
    image_n= cv2.resize(image_n,dsize=(img_h, img_w),interpolation=cv2.INTER_CUBIC)
    try:
      image_n = image_n[:, :, :3]
    except:
      continue
  
    xtrain[index] = preprocess_input(np.array(image_n))
    for i in images_npy[index][1]:
        x1=int(i["points"][0]['x']*img_w)
        x2=int(i["points"][1]['x']*img_w)
        y1=int(i["points"][0]['y']*img_h)
        y2=int(i["points"][1]['y']*img_h)
        masks[index][y1:y2, x1:x2]=1


### C. Split the data into train and test[400:9]. [1 Marks]

In [ ]:
print(xtrain.shape)
print(masks.shape)

### D. Select random image from the train data and display original image and masked image. [2 Marks]

In [ ]:
X=images_npy[:,0]
Y=images_npy[:,1]
from sklearn.model_selection import train_test_split

# split into train test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.022)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


In [ ]:
#Function to plot random image along with its masked image
j=np.random.randint(0,399)
for index in range(X_train.shape[0]):
    img = images_npy[index][0]
    img = cv2.resize(img, dsize=(200, 200), interpolation=cv2.INTER_CUBIC)
original_image=X_train[j]
plt.imshow(original_image/255,cmap=plt.cm.binary)
def detect_mask():
  face_cascade = cv2.CascadeClassifier('Desktop/data/haarcascade_frontalface_default.xml')
  global j
  
  
  #get the cordinates for the human face mask
  
  face_rects = face_cascade.detectMultiScale(X_train[j]) 
    
  for (x,y,w,h) in face_rects: 
      cv2.rectangle(X_train[j], (x,y), (x+w,y+h), (255,255,255), 5)
      plt.imshow(X_train[j])
      
detect_mask() 

In [ ]:
plt.imshow(xtrain[0])

## Q2. Model building [11 Marks]

### A. Design a face mask detection model. [4 Marks]

In [ ]:
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.layers import Concatenate, UpSampling2D, Conv2D, Reshape
from tensorflow.keras.models import Model
import tensorflow as tf

height_c= 28
width_c= 28
batch= 1


In [ ]:
mobilenet = MobileNet(input_shape=(img_h,img_w,3), include_top=False, alpha=alpha, weights="imagenet")
mobilenet.summary()

In [ ]:
def create_model(trainable=False):
    mobilenet = MobileNet(input_shape=(img_h,img_w,3), include_top=False, alpha=alpha, weights="imagenet")

    for layer in mobilenet.layers:
        layer.trainable = trainable
        
    block0= mobilenet.layers[0].input
    block1 =mobilenet.get_layer("conv_pw_1_relu").output
    block2 =mobilenet.get_layer("conv_pw_2_relu").output
    block3 =mobilenet.get_layer("conv_pw_3_relu").output
    block5 =mobilenet.get_layer("conv_pw_5_relu").output
    block11 =mobilenet.get_layer("conv_pw_11_relu").output
    block13 =mobilenet.get_layer("conv_pw_13_relu").output


    dB= Concatenate()([UpSampling2D()(block13), block11])
    dB= Concatenate()([UpSampling2D()(dB), block5])
    dB= Concatenate()([UpSampling2D()(dB), block3])
    dB= Concatenate()([UpSampling2D()(dB), block1])
    dB= Concatenate()([UpSampling2D()(dB), block0])

    dB= Conv2D(1, kernel_size=1, activation="sigmoid")(dB)
    dB= Reshape((img_h,img_w))(dB)

    return Model(inputs=mobilenet.input, outputs=dB) 

In [ ]:
mobilenet=create_model()
mobilenet.summary()


### B. Design your own Dice Coefficient and Loss function. [2 Marks]

In [ ]:
def dice_coff(y_true, y_pred):
    num = 2 * tf.reduce_sum(y_true * y_pred)
    denom = tf.reduce_sum(y_true + y_pred)

    return num / (denom + tf.keras.backend.epsilon())

### C. Train and tune the model as required. [3 Marks]

In [ ]:
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.backend import log, epsilon
def loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) - log(dice_coff(y_true, y_pred) + epsilon())

In [ ]:
model = create_model(False)
model.summary()
optimizer = tf.keras.optimizers.Adam()
model.compile(loss=loss, optimizer = optimizer, metrics=[dice_coff])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
checkpoint = ModelCheckpoint("model-{loss:.2f}.h5", monitor="loss", verbose=1, save_best_only=True,
                             save_weights_only=True, mode="min", save_freq=1)
stop = EarlyStopping(monitor="loss", patience=5, mode="min")
reduce_lr = ReduceLROnPlateau(monitor="loss", factor=0.2, patience=1, min_lr=1e-6, verbose=1, mode="min")

In [ ]:
model.fit(xtrain,masks,epochs = 1,verbose=1,batch_size=3,callbacks=[checkpoint,reduce_lr,stop])

### D. Evaluate and share insights on performance of the model. [2 Marks]

Model Mobilenet
Build with 992 training images of 



Parameters for evalution 

Loss--- During traing, loss perfromance is getting improved, from around 3.4 to 1.08.
The early stopping save from stucking into local minima.
Dice coefficient is also improving ( from 0.14 to 0.33), which will improve the pixel-wise prediction efficiency.

In [ ]:
image_test=images_npy[400][0]
plt.imshow(image_test)


In [ ]:
sample_image = image_test
image = cv2.resize(sample_image, dsize=(img_h,img_w), interpolation=cv2.INTER_CUBIC)
feat_scaled = preprocess_input(np.array(image, dtype=np.float32))

pred_mask = cv2.resize(1.0*(model.predict(x=np.array([feat_scaled]))[0] > 0.2), (img_w,img_h))

image2 = image
image2[:,:,0] = pred_mask*image[:,:,0]
image2[:,:,1] = pred_mask*image[:,:,1]
image2[:,:,2] = pred_mask*image[:,:,2]

out_image = image2

plt.imshow(out_image)

In [ ]:
plt.imshow(pred_mask, alpha=1)

## Q3. Test the model predictions on the test image: ‘image with index 3 in the test data’ and visualise the predicted masks on the faces in the image. [2 Marks]

In [ ]:
sample_image = image_test
image = cv2.resize(sample_image, dsize=(img_h,img_w), interpolation=cv2.INTER_CUBIC)
feat_scaled = preprocess_input(np.array(image, dtype=np.float32))

pred_mask = cv2.resize(1.0*(model.predict(x=np.array([feat_scaled]))[0] > 0.2), (img_w,img_h))

image2 = image
image2[:,:,0] = pred_mask*image[:,:,0]
image2[:,:,1] = pred_mask*image[:,:,1]
image2[:,:,2] = pred_mask*image[:,:,2]

out_image = image2

plt.imshow(out_image)

In [ ]:
plt.imshow(pred_mask, alpha=1)


=============== PART B  ======================

### Create an image dataset to be used by AI team build an image classifier data. 

### Profile images of people are given.

## Q1. Read/import images from folder ‘training_images’. [2 Marks]

In [ ]:
import zipfile

## Q2. Write a loop which will iterate through all the images in the ‘training_images’ folder and detect the faces present on all the images. [3 Marks]


## Q3. From the same loop above, extract metadata of the faces and write into a DataFrame. [3 Marks]

## Q4. Save the output Dataframe in .csv format. [2 Marks]

In [ ]:
def mask_csv(path):
    import os
    import cv2
    import matplotlib.pyplot as plt
    import pandas as pd
    import numpy as np
        
    face=cv2.CascadeClassifier("Desktop/data/haarcascade_frontalface_default.xml")
    li = os.listdir(path)
    name_of_img = []
    no_of_img = []
    x=[]
    y=[]
    h=[]
    w=[]

    for i in range(len(li)):    
        img = cv2.imread(path+li[i])
#         img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        # detect the number of face and quardinates
        
        find_faces = face.detectMultiScale(img,1.1,20)
        for (x1,y1,w1,h1) in find_faces:
            
            
            name_of_img.append(li[i])
            no_of_img.append(len(find_faces))
            
            x.append(x1)
            y.append(y1)
            w.append(w1)
            h.append(h1)
            
            img1 = cv2.rectangle(img,(x1,y1),(x1+w1,y1+h1),(127,0,205),3)
            img1= cv2.cvtColor(img1,cv2.COLOR_BGR2RGB)

        plt.matshow(img1)       
        
    data_dict = {'x':x,'y':y, 'w':w,'h':h,'no_of_img':no_of_img,'name_of_img':name_of_img}

    df = pd.DataFrame(data_dict)
    df.to_csv('test.csv')
    return df 

In [ ]:
mask_csv("Desktop/data/training_images/")

=============== PART C  ======================

In [ ]:
import zipfile

## Q1. Unzip, read and Load data(‘PINS.zip’) into session. [2 Marks]

## Q2. Write function to create metadata of the image. [4 Marks]

## Q3. Write a loop to iterate through each and every image and create metadata for all the images. [4 Marks]

In [ ]:
import numpy as np
import os

class IdentityMetadata():
    def __init__(self, base, name, file):
       
        self.base = base
        
        self.name = name
        
        self.file = file

    def __repr__(self):
        return self.image_path()

    def image_path(self):
        return os.path.join(self.base, self.name, self.file) 
    
def load_metadata(path):
    metadata = []
    for i in os.listdir(path):
        for f in os.listdir(os.path.join(path, i)):
           
            ext = os.path.splitext(f)[1]
            if ext == '.jpg' or ext == '.jpeg':
                metadata.append(IdentityMetadata(path, i, f))
    return np.array(metadata)
metadata = load_metadata('Desktop/data/PINS')


In [ ]:
metadata[0]

In [ ]:
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import ZeroPadding2D, Convolution2D, MaxPooling2D, Dropout, Flatten, Activation

def vgg_face():	
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
    model.add(Convolution2D(64, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    
    model.add(Convolution2D(4096, (7, 7), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Convolution2D(4096, (1, 1), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Convolution2D(2622, (1, 1)))
    model.add(Flatten())
    model.add(Activation('softmax'))
    return model

In [ ]:
model=vgg_face()
model.load_weights('Desktop/data/vgg_face_weights.h5')

In [ ]:
from tensorflow.keras.models import Model
vgg_face_descriptor = Model(inputs=model.layers[0].input, outputs=model.layers[-2].output)

In [ ]:
type(vgg_face_descriptor)

In [ ]:
vgg_face_descriptor.inputs, vgg_face_descriptor.outputs

## Q4. Generate Embeddings vectors on the each face in the dataset. [4 Marks]

## Q5. Build distance metrics for identifying the distance between two similar and dissimilar images. [4 Marks]

In [ ]:
import cv2
def load_image(path):
    img = cv2.imread(path, 1)
    return img[...,::-1]

In [ ]:
total_images = len(metadata)

In [ ]:
metadata[0]

In [ ]:
embeddings = np.zeros((metadata.shape[0], 2622))
for i, m in enumerate(metadata):
    img_path = metadata[i].image_path()
    img = load_image(img_path)
    img = (img / 255.).astype(np.float32)
    img = cv2.resize(img, dsize = (224,224))
    embedding_vector = vgg_face_descriptor.predict(np.expand_dims(img, axis=0))[0]
    embeddings[i]=embedding_vector
    

In [ ]:
embeddings[1][6]

In [ ]:
def distance(emb1, emb2):
    return np.sum(np.square(emb1 - emb2))

In [ ]:
import matplotlib.pyplot as plt
def show_pair(idx1, idx2):
    plt.figure(figsize=(8,3))
    plt.suptitle(f'Distance between {idx1} & {idx2}= {distance(embeddings[idx1], embeddings[idx2]):.2f}')
    plt.subplot(121)
    plt.imshow(load_image(metadata[idx1].image_path()))
    plt.subplot(122)
    plt.imshow(load_image(metadata[idx2].image_path()));    

show_pair(40, 41)
show_pair(20, 60)

In [ ]:
train_idx = np.arange(metadata.shape[0]) % 9 != 0     
test_idx = np.arange(metadata.shape[0]) % 9 == 0


X_train = embeddings[train_idx]


X_test = embeddings[test_idx]
targets = np.array([m.name for m in metadata])


y_train = targets[train_idx]


y_test = targets[test_idx]


In [ ]:
y_test[0], y_train[72]

In [ ]:
len(np.unique(y_test)), len(np.unique(y_train))

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)

In [ ]:
print(le.classes_)
y_test_encoded = le.transform(y_test)

In [ ]:
y_test_encoded

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

## Q6. Use PCA for dimensionality reduction. [2 Marks]

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=128)
X_train_pca = pca.fit_transform(X_train_std)
X_test_pca = pca.transform(X_test_std)

## Q7. Build an SVM classifier in order to map each image to its right person. [4 Marks]

In [ ]:
from sklearn.svm import SVC

clf = SVC(C=5., gamma=0.001)
clf.fit(X_train_pca, y_train_encoded)

In [ ]:
y_predict = clf.predict(X_test_pca)

In [ ]:
y_predict_encoded = le.inverse_transform(y_predict)

In [ ]:
y_test_encoded[32:49]

In [ ]:
from sklearn.metrics import precision_recall_curve,accuracy_score,f1_score,precision_score,recall_score
accuracy_score(y_test_encoded, y_predict)


## Q8. Import and display the the test images. [2 Marks]

In [ ]:
img_path = metadata[11].image_path()
img = load_image("Desktop/data/Benedict_Cumberbatch9.jpg")


In [ ]:
plt.imshow(img)

In [ ]:
img_path = metadata[0].image_path()
img2= load_image("Desktop/data/Dwayne_Johnson4.jpg")


In [ ]:
plt.imshow(img2)

In [ ]:
path_Rock="Desktop/data/PINS1/PINS1/PINS1/pins_Dwayne_Johnson/Dwayne_Johnson4.jpg"
for i in range(len(metadata)):
  if str(metadata[i])=="Desktop/data/Dwayne_Johnson4.jpg":
    print(i)

In [ ]:
plt.imshow(load_image(metadata[580].image_path()))

In [ ]:
path_DrStrange="Desktop/data/PINS1/PINS1/PINS1/pins_Benedict_Cumberbatch/Benedict_Cumberbatch9.jpg"
for i in range(len(metadata)):
  if str(metadata[i])=="Desktop/data/PINS1/PINS1/PINS1/pins_Benedict_Cumberbatch/Benedict_Cumberbatch9.jpg":
    print(i)

In [ ]:
plt.imshow(load_image(metadata[1052].image_path()))

## Q9. Use the trained SVM model to predict the face on both test images. [4 Marks]

In [ ]:
#Test Image DrStrange
test_idx =580

test_image = load_image(metadata[test_idx].image_path())
test_prediction = y_predict[test_idx]
test_identity =  y_predict_encoded[test_idx]

plt.imshow(test_image)
plt.title(f"Celebrity's name is {test_identity}");


In [ ]:
#Test Image Rock
test_idx =1052

test_image = load_image(metadata[test_idx].image_path())
test_prediction = y_predict[test_idx]
test_identity =  y_predict_encoded[test_idx]

plt.imshow(test_image)
plt.title(f"Celebrity's name is {test_identity}");